### Summary
For my final project I have chosen to compete in a kaggle competition. <br>
The competition is being put on by HM clothing and the goal is to predict what shoppers will buy in the next 7 days.

The competition details are here -> https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations


### Data

The data comprises of 4 files:
1. articles.csv - detailed metadata for each article_id available for purchase
2. sample_submission.csv - a sample submission file in the correct format
3. customers.csv - metadata for each customer_id in dataset
4. transactions_train.csv - the training data, consisting of the purchases each customer for each date, as well as additional information. Duplicate rows correspond to multiple purchases of the same item. Your task is to predict the article_ids each customer will purchase during the 7-day period immediately after the training data period.

### Data Strategy

My strategy is to first clean the data so that I can feed multiple days of shopping data into the model. I will see if I can cluster shoppers into categories. I will see if I can cluster items into categories. I think that it will be easier to first predict that the shopper will buy a clothing type, like shoes, then predict the type of shoe rather than trying to predict the exact shoe first. 

In [2]:
import pandas as pd
import numpy as np
import sklearn as sl
import scipy as sp
from pandas_profiling import ProfileReport

In [11]:
trainingdata = pd.read_csv('data/transactions_train.csv',nrows = 500000)
articlesdata = pd.read_csv('data/articles.csv')
customersdata = pd.read_csv('data/customers.csv')


In [12]:
merged_set = trainingdata.merge(articlesdata, left_on='article_id', right_on='article_id',
          suffixes=('_left', '_right'))
merged_set = merged_set.merge(customersdata, left_on='customer_id', right_on='customer_id',
          suffixes=('_left', '_right'))

In [13]:
merged_set.drop(columns=['FN','Active'])

,t_dat,customer_id,article_id,price,sales_channel_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,...,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,club_member_status,fashion_news_frequency,age,postal_code
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,663713,Atlanta Push Body Harlow,283,Underwear body,Underwear,...,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up body with underwired, moulded, pa...",ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
1,2018-09-24,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,663713,Atlanta Push Body Harlow,283,Underwear body,Underwear,...,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up body with underwired, moulded, pa...",ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
2,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,541518,Rae Push (Melbourne) 2p,306,Bra,Underwear,...,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up bras with underwired, moulded, pa...",ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,2018-09-20,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,663713001,0.049475,2,663713,Atlanta Push Body Harlow,283,Underwear body,Underwear,...,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Lace push-up body with underwired, moulded, pa...",ACTIVE,Regularly,30.0,aea9af1fd9b5d3aa98820e66a7a093f54def0e4db44153...
4,2018-09-20,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,537346026,0.041237,2,537346,Ridge,252,Sweater,Garment Upper body,...,Ladieswear,15,Womens Everyday Collection,1003,Knitwear,Oversized jumper in a soft knit with a deep V-...,ACTIVE,Regularly,30.0,aea9af1fd9b5d3aa98820e66a7a093f54def0e4db44153...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,2018-09-29,94283ab4ac370f4c8d9220f21513858b6318d27798f343...,683171001,0.027441,2,683171,KOMPIS 3p ls,255,T-shirt,Garment Upper body,...,Baby/Children,79,Girls Underwear & Basics,1002,Jersey Basic,Long-sleeved tops in lightweight organic cotto...,ACTIVE,Regularly,34.0,48a4dc4905c7dd9ecdd84e002857b5dc2c13bb05d5f9e1...
499996,2018-09-29,9570244d3fbfa4a5003324bd5488457248ec5de10b563d...,304576027,0.002542,1,304576,FORTUNA T-SHIRT,255,T-shirt,Garment Upper body,...,Menswear,20,Contemporary Smart,1005,Jersey Fancy,Jersey T-shirt,ACTIVE,Regularly,23.0,89fc2d3d103833d2bb60700aae18e7f1613010aec897c8...
499997,2018-09-29,967f2414a612c4468fbd889abae7d29c2be2e8f459b7ce...,468345021,0.008119,1,468345,PETER SLIM FIT POLO,257,Polo shirt,Garment Upper body,...,Menswear,26,Men Underwear,1002,Jersey Basic,"Short-sleeved polo shirt in stretch, cotton-bl...",ACTIVE,Regularly,28.0,486afc7a04933f5d8d6a0197b97cd0fd1f1958094dd6dc...
499998,2018-09-29,972b7e1ffe7f6e722500806d7d307671bdae97e0af825a...,538592002,0.015237,2,538592,Spider velour body,253,Vest top,Garment Upper body,...,Divided,53,Divided Collection,1005,Jersey Fancy,"V-neck body in velour with narrow, double shou...",ACTIVE,NONE,34.0,6596fe332ad79f7ab139f7d1cbcd824fcadb58983127ab...


In [14]:
#profile = ProfileReport(merged_set, title="Pandas Profiling Report", explorative=True)

In [15]:
#profile.to_notebook_iframe()

In [24]:
#pivoted_merged_data = merged_set.pivot_table(index='customer_id', columns='t_dat', values='article_id', aggfunc=lambda x: ' '.join(x))
pivoted_merged_data = merged_set.pivot_table(index=['customer_id'],
                             columns=['t_dat'],
                             values=['article_id'],
                             aggfunc=lambda x: ' '.join(str(v) for v in x))


In [26]:
pivoted_merged_data.head()

article_id  \
t_dat                                                                                      2018-09-20   
customer_id                                                                                             
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...                                                NaN   
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...                                663713001 541518023   
00007d2de826758b65a93dd24ce629ed66842531df66993...  505221004 685687003 685687004 685687001 505221001   
0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c919...                                                NaN   
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d6...                                                NaN   

                                                                                   \
t_dat                                                                  2018-09-21   
customer_id                                                                         
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...  640244003 583558001 639677008   
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...                            NaN   
00007d2de826758b65a93dd24ce629ed66842531df66993...                            NaN   
0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c919...                            NaN   
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d6...                            NaN   

                                                               \
t_dat                                              2018-09-22   
customer_id                                                     
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...        NaN   
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...        NaN   
00007d2de826758b65a93dd24ce629ed66842531df66993...        NaN   
0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c919...        NaN   
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d6...        NaN   

                                                                                             \
t_dat                                                                            2018-09-23   
customer_id                                                                                   
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...                                      NaN   
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...                                      NaN   
00007d2de826758b65a93dd24ce629ed66842531df66993...  567618001 508184022 605106001 522992001   
0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c919...                                      NaN   
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d6...                                      NaN   

                                                               \
t_dat                                              2018-09-24   
customer_id                                                     
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...        NaN   
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...  663713001   
00007d2de826758b65a93dd24ce629ed66842531df66993...        NaN   
0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c919...        NaN   
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d6...        NaN   

                                                                         \
t_dat                                                        2018-09-25   
customer_id                                                               
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee...  521269001 666448006   
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad...                  NaN   
00007d2de826758b65a93dd24ce629ed66842531df66993...                  NaN   
0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c919...                  NaN   
00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d6...                  NaN   

                                                               \
t_dat                                              2018-09-26   
customer_id                        

In [23]:
pivoted_merged_data.columns

MultiIndex([('article_id', '2018-09-20'),
            ('article_id', '2018-09-21'),
            ('article_id', '2018-09-22'),
            ('article_id', '2018-09-23'),
            ('article_id', '2018-09-24'),
            ('article_id', '2018-09-25'),
            ('article_id', '2018-09-26'),
            ('article_id', '2018-09-27'),
            ('article_id', '2018-09-28'),
            ('article_id', '2018-09-29')],
           names=[None, 't_dat'])